# **Generating audio from my pre-trained model**

In [0]:
import torch
from torch import autograd
from torch.utils.data import DataLoader
from torch import optim
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import pickle as pk

## **generator**

In [0]:
class Generator(nn.Module):
    def __init__(self, model_size=64, num_channels=1, latent_dim=100,
                 post_proc_filt_len=512):
        super(Generator, self).__init__()
        self.model_size = model_size # d
        self.num_channels = num_channels # c
        self.latent_dim = latent_dim
        self.post_proc_filt_len = post_proc_filt_len
        
        self.fc1 = nn.DataParallel(nn.Linear(latent_dim, 256 * model_size))
        
        self.tconv1 = None
        self.tconv2 = None
        self.tconv3 = None
        self.tconv4 = None
        self.tconv5 = None
        

        self.tconv1 = nn.DataParallel(
                 nn.ConvTranspose1d(16 * model_size, 8 * model_size, 25, stride=4, padding=11,
                                    output_padding=1))
        self.tconv2 = nn.DataParallel(
                 nn.ConvTranspose1d(8 * model_size, 4 * model_size, 25, stride=4, padding=11,
                                    output_padding=1))
        self.tconv3 = nn.DataParallel(
                 nn.ConvTranspose1d(4 * model_size, 2 * model_size, 25, stride=4, padding=11,
                                    output_padding=1))
        self.tconv4 = nn.DataParallel(
                 nn.ConvTranspose1d(2 * model_size, model_size, 25, stride=4, padding=11,
                                    output_padding=1))
        self.tconv5 = nn.DataParallel(
                 nn.ConvTranspose1d(model_size, num_channels, 25, stride=4, padding=11,
                                    output_padding=1))

        
        if post_proc_filt_len:
            self.ppfilter1 = nn.DataParallel(nn.Conv1d(num_channels, num_channels, post_proc_filt_len))
        
        for m in self.modules():
            if isinstance(m, nn.ConvTranspose1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)

    def forward(self, x):

        x = self.fc1(x).view(-1, 16 * self.model_size, 16)
        x = F.relu(x)
        output = None
        
        x = F.relu(self.tconv1(x))
        x = F.relu(self.tconv2(x))
        x = F.relu(self.tconv3(x))
        x = F.relu(self.tconv4(x))
        output = torch.tanh(self.tconv5(x))
                    
        if self.post_proc_filt_len:
            # Pad for "same" filtering
            if (self.post_proc_filt_len % 2) == 0:
                pad_left = self.post_proc_filt_len // 2
                pad_right = pad_left - 1
            else:
                pad_left = (self.post_proc_filt_len - 1) // 2
                pad_right = pad_left
            output = self.ppfilter1(F.pad(output, (pad_left, pad_right)))
        
        return output

## **save model and save audio**

In [0]:

def save_sample(data,sample_size,save_path):
  for i in range(sample_size):
    sample = data[i].reshape(16384,1)    
    
    librosa.output.write_wav(save_path+'{}.wav'.format(i),sample, 16000)

def showing_wave(data):
  sample = data.reshape(16384)
  print(sample)
  plt.figure(figsize=(25,8))
  librosa.display.waveplot(sample,16000)
  plt.show()
  plt.close()
    

In [0]:
if __name__=="__main__":
 
    laten_dim = 100
    sample_size = 2  
    load_model = True
    G = Generator()
    
    #try to change the path
    file_path = './drive/My Drive/bird1.pkl'
    save_path = './home/'
    #load the parameter of network 
    
    print('loading model...')
      
    
    G.load_state_dict(torch.load(file_path,map_location='cpu')) 
  
    
    #========= test generator ==============#
    #sample noise
    sample_noise = torch.randn(sample_size, laten_dim) 
    sample_output = G(sample_noise)
    for j in range(sample_size):
      showing_wave(sample_output.data.numpy()[j])
    save_sample(sample_output.data.numpy(),sample_size,save_path)
    
    